In [1]:
import pandas as pd

dataset = pd.read_csv("data1.csv")
dataset["risk_score"] = dataset["risk_score"] / dataset["risk_score"].max()
dataset.head(5)

,text,label,risk_score
0,Предоставление права использования КИП в пред...,Если Пользователь не успеет зарегистрировать д...,0.571429
1,Предоставление права использования КИП в пред...,Если Пользователь не успеет зарегистрировать д...,0.571429
2,Предоставление права использования КИП в пред...,Если Пользователь не успеет зарегистрировать д...,0.571429
3,г. Москва,neutral,0.000000
4,"ООО «Киберклуб», именуемое в дальнейшем Правоо...",neutral,0.000000


In [2]:
dataset.shape

(65, 3)

In [3]:
import re

def text_processing(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'(\d)\s+(?=\d)', r'\1', text)
    return text

def clean_label(label):
    if pd.isna(label):
        return 'unknown'
    
    cleaned = str(label).strip().replace('"', '').replace('”', '')

    typo_corrections = {
        'nol_risk': 'no_risk',
        'gal_risk': 'legal_risk',
    }
    
    return typo_corrections.get(cleaned, cleaned)

dataset["text"] = dataset["text"].apply(text_processing)
# dataset['label'] = dataset['label'].apply(clean_label)
# dataset = dataset[dataset["label"] != "unknown"]

In [4]:
dataset.head(5)

,text,label,risk_score
0,предоставление права использования кип в пред...,Если Пользователь не успеет зарегистрировать д...,0.571429
1,предоставление права использования кип в пред...,Если Пользователь не успеет зарегистрировать д...,0.571429
2,предоставление права использования кип в пред...,Если Пользователь не успеет зарегистрировать д...,0.571429
3,г москва,neutral,0.000000
4,ооо киберклуб именуемое в дальнейшем правообла...,neutral,0.000000


In [5]:
from sklearn.model_selection import train_test_split

# train_data, val_data = train_test_split(dataset, test_size=0.2, stratify=dataset['label'], random_state=32)
train_data = dataset
X_train_text = train_data["text"]
y_train = train_data["label"]
X_train_score = train_data["risk_score"]

# X_val_text = val_data["text"]
# y_val = val_data["label"]
# X_val_score = val_data["risk_score"]


In [6]:
from transformers import BertModel, BertTokenizer, BertConfig, BertPreTrainedModel
import torch.nn as nn
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class DualBert(BertPreTrainedModel):
    def __init__(self, config, num_classes):
        super().__init__(config)
        self.bert = BertModel(config)
        self.classifier = nn.Linear(config.hidden_size, num_classes)
        self.regressor = nn.Linear(config.hidden_size, 1)
        self.init_weights()
    
    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask)
        pooled_output = outputs.pooler_output
        
        logits = self.classifier(pooled_output)
        risk_score = self.regressor(pooled_output).squeeze(-1)
        
        return logits, risk_score

def loss_fn(logits, risk_pred, labels, risk_score, alpha=0.4):
    clf_fn = nn.CrossEntropyLoss()
    reg_fn = nn.MSELoss()
    
    loss_clf = clf_fn(logits, labels)
    loss_reg = reg_fn(risk_pred, risk_score)
    
    total_loss = alpha * loss_clf + (1 - alpha) * loss_reg

    return total_loss


model_id = "DeepPavlov/rubert-base-cased"

unique_classes = dataset["label"].unique()

num_labels = len(unique_classes)

tokenizer = BertTokenizer.from_pretrained(model_id)
config = BertConfig.from_pretrained(model_id, num_labels=num_labels)
model = DualBert.from_pretrained(model_id, config=config, num_classes=num_labels).to(device)


c:\Users\vdoff\Desktop\law-exp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of DualBert were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.cuda()

DualBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [8]:
unique_classes

array(['Если Пользователь не успеет зарегистрировать договор до 15 апреля 2025 г., договор считается незаключенным (ст. 1028 ГК РФ). Риск: действия Пользователя будут незаконными, включая использование товарного знака.',
       'neutral',
       'Запрет открывать клубы в радиусе 15 км в течение 5 лет может быть признано ограничивающим  конкуренцию (п. 3 ст. 1033 ГК РФ).',
       'Установление фиксированных цен Правообладателем нарушает ст. 1033 ГК РФ, которая запрещает антиконкурентные условия. Это может привести к спорам с антимонопольной службой.',
       'Требование передавать базы данных клиентов без их согласия нарушает закон № 152-ФЗ о персональных данных. Пользователь рискует штрафами до 75 000 руб. (ст. 13.11 КоАП РФ).',
       'Указание «10% от выручки» без уточнения (валовая/чистая) противоречит ст. 424 ГК РФ (цена должна быть определена четко). Суд может признать условие несогласованным.',
       'Уменьшение гибкости ведения бизнеса, хотя такой запрет не противоречит законод

In [9]:
X_train_text = tokenizer(X_train_text.tolist(), max_length=256, padding=True, truncation=True, return_tensors="pt")
# X_val_text = tokenizer(X_val_text.tolist(), max_length=256, padding=True, truncation=True, return_tensors="pt")

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(unique_classes)
y_train = label_encoder.transform(y_train)
# y_val = label_encoder.transform(y_val)

In [11]:
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, inputs, labels, masks, risk_scores):
        self.inputs = inputs
        self.labels = labels
        self.mask = masks
        self.risk_scores = risk_scores

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return {
            "input_ids": torch.tensor(self.inputs[index]),
            "attention_mask": torch.tensor(self.mask[index]),
            "labels": torch.tensor(self.labels[index]),
            "risk_scores": torch.tensor(self.risk_scores[index])
        }

train_dataset = TextDataset(X_train_text["input_ids"], y_train, X_train_text["attention_mask"], X_train_score.tolist())
# val_dataset = TextDataset(X_val_text["input_ids"], y_val, X_val_text["attention_mask"], X_val_score.tolist())

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [12]:
from tqdm import tqdm
import numpy as np


def train(model, optimizer, loss_fn, train_loader, lr=1e-5, num_epochs=15):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        epoch_losses = []
        if num_epochs > 8:
            optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)

        for batch in tqdm(train_loader, desc=f"Training epoch: {epoch}"):
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device)
            }
            labels = batch["labels"].to(device)
            risk_scores = batch["risk_scores"].to(device)
            model.zero_grad()
            logits, pred = model(**inputs)
            loss = loss_fn(logits, pred, labels, risk_scores)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            epoch_losses.append(loss.item())

        train_acc, train_mse, _ = eval(model, loss_fn, train_loader)
        # val_acc, val_mse, val_loss = eval(model, loss_fn, val_loader)

        ep_loss = sum(epoch_losses) / len(epoch_losses)
        print(f"Train loss: {ep_loss}, accuracy on train: {train_acc}, mse on train: {train_mse}")
        # print(f"Val loss: {val_loss}, accuracy on val: {val_acc}, mse on val: {val_mse}")

        # if val_acc > best_val_acc and epoch > 6:
        #     best_val_acc = val_acc
        #     torch.save(model.state_dict(), "best_model.pth")

    return model

def eval(model, loss_fn,data_loader):
    model.eval()
    correct_clf = 0
    total_clf = 0
    total_loss = 0

    risk_preds = []
    risk_targets = []
    
    with torch.no_grad():
        for batch in data_loader:
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device)
            }
            labels = batch["labels"].to(device)
            risk_scores = batch["risk_scores"].to(device)
            
            logits, risk_pred = model(**inputs)
            
            preds = torch.argmax(logits, dim=1)
            correct_clf += (preds == labels).sum().item()
            total_clf += labels.size(0)
            
            risk_preds.extend(risk_pred.cpu().numpy())
            risk_targets.extend(risk_scores.cpu().numpy())
            loss = loss_fn(logits, risk_pred, labels, risk_scores)
            
            total_loss += loss.item()
    
    clf_accuracy = correct_clf / total_clf
    
    risk_preds = np.array(risk_preds)
    risk_targets = np.array(risk_targets)
    

    mse = np.mean((risk_preds - risk_targets)**2)
    
    return clf_accuracy, mse, total_loss

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)

model = train(model, optimizer, loss_fn, train_loader, lr=2e-5, num_epochs=40)

Training epoch: 0:   0%|          | 0/3 [00:00<?, ?it/s]C:\Users\vdoff\AppData\Local\Temp\ipykernel_25176\1864683238.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(self.inputs[index]),
C:\Users\vdoff\AppData\Local\Temp\ipykernel_25176\1864683238.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(self.mask[index]),
Training epoch: 0: 100%|██████████| 3/3 [00:02<00:00,  1.31it/s]


Train loss: 1.1124444802602131, accuracy on train: 0.2153846153846154, mse on train: 0.11699166148900986


Training epoch: 1: 100%|██████████| 3/3 [00:01<00:00,  1.51it/s]


Train loss: 0.9725470940272013, accuracy on train: 0.46153846153846156, mse on train: 0.13222892582416534


Training epoch: 2: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]


...

Train loss: 0.06392987569173177, accuracy on train: 1.0, mse on train: 0.011593868024647236


Training epoch: 38: 100%|██████████| 3/3 [00:01<00:00,  1.65it/s]


Train loss: 0.051768604665994644, accuracy on train: 1.0, mse on train: 0.014717102982103825


Training epoch: 39: 100%|██████████| 3/3 [00:01<00:00,  1.65it/s]


Train loss: 0.07717095439632733, accuracy on train: 1.0, mse on train: 0.013753755949437618


In [14]:
torch.save(model.state_dict(), "best_model.pth")

In [15]:
def test(model, tokenizer, text):
    input = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors="pt", return_token_type_ids=False).to(device)
    logits, pred = model(**input)
    return logits, pred


model.load_state_dict(torch.load('best_model.pth'))

texts = [
    "Любая из сторон во всякое время вправе досрочно отказаться от договора, уведомив об этом другую сторону не позднее чем за тридцать дней и уплатив ей отступное в размере 360000 руб.",
    "За несвоевременную подачу документов на регистрацию в Роспатент Правообладатель выплачивает Пользователю штраф в размере 1000 руб. за каждый день просрочки.",
    "Правообладатель вправе каждый день приходить в киберклуб"
]


for text in texts:
    text = text_processing(text)

    print(text)

    out, pred = test(model, tokenizer, text)
    pred_label_id = torch.argmax(out, dim=1)
    pred_label_id.cpu()
    label = label_encoder.inverse_transform([pred_label_id.item()])[0]
    print(label, f"{pred.item():.2f}")

любая из сторон во всякое время вправе досрочно отказаться от договора уведомив об этом другую сторону не позднее чем за тридцать дней и уплатив ей отступное в размере 360000 руб
neutral 0.17
за несвоевременную подачу документов на регистрацию в роспатент правообладатель выплачивает пользователю штраф в размере 1000 руб за каждый день просрочки
neutral 0.04
правообладатель вправе каждый день приходить в киберклуб
neutral 0.04
